In [1]:
import os
import asyncio
import aiohttp
import json
import re
from random import shuffle
import random
import pandas as pd
from dataclasses import dataclass

In [2]:
from card_utils import read_all_game_data, Card, PlayerCards, PlayerStats

In [3]:
random.seed(42)

In [4]:
from deck_rate_fetcher import fetch_deck_rating

In [5]:
import dotenv
dotenv.load_dotenv()

import nest_asyncio
nest_asyncio.apply()

In [6]:
from langchain_openai import AzureChatOpenAI
from openai import AsyncAzureOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [7]:
game_cards_list, player_cards_list, player_stats, player_tags_list = read_all_game_data()

In [8]:
top_decks_per_card = pd.read_json("data/top_decks_per_card.json").to_dict()

In [9]:
@dataclass
class LLMModel:
    name: str
    instance: AzureChatOpenAI | AsyncAzureOpenAI

In [10]:
llm_gpt3 = LLMModel(
    name="gpt-35-turbo",
    instance=AzureChatOpenAI(
        model_name="gpt-35-turbo",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
)

llm_gpt4o = LLMModel(
    name="gpt-4o",
    instance=AzureChatOpenAI(
        model_name="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
)

llm_gpt5 = LLMModel(
    name="gpt-5-chat",
    instance=AzureChatOpenAI(
        model_name="gpt-5-chat",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
)

In [11]:
class SimpleAIMessage:
    """Clase simple para empaquetar la respuesta con un atributo .content"""
    def __init__(self, content):
        self.content = content

# Adaptador para AsyncAzureOpenAI de openai
class OpenAIClientAdapter:
    def __init__(self, client: AsyncAzureOpenAI, model_name: str, temperature: float = 0):
        self.client = client
        self.model_name = model_name
        self.temperature = temperature
        print(f"Adaptador creado para el modelo: {self.model_name}")

    def _convert_lc_messages_to_dict(self, messages):
        output_messages = []
        for msg in messages:
            role = msg.type
            if role == "human":
                role = "user"
            
            output_messages.append({"role": role, "content": msg.content})
        return output_messages

    async def ainvoke(self, messages):
        try:
            dict_messages = self._convert_lc_messages_to_dict(messages)
            
            response = await self.client.chat.completions.create(
                model=self.model_name,
                messages=dict_messages,
                temperature=self.temperature
            )
            
            content = response.choices[0].message.content
            
            return SimpleAIMessage(content=content)
        
        except Exception as e:
            print(f"Error en el adaptador de OpenAI ({self.model_name}): {e}")
            return SimpleAIMessage(content=None)

In [12]:
print("Creando clientes y adaptadores...")

async_openai_client = AsyncAzureOpenAI(
    azure_endpoint=dotenv.get_key(dotenv.find_dotenv(), "AZURE_OPENAI_ENDPOINT"),
    api_key=dotenv.get_key(dotenv.find_dotenv(), "AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview"
)

llm_grok_adapter = LLMModel(
    name="grok-4-fast-non-reasoning",
    instance=OpenAIClientAdapter(
        client=async_openai_client,
        model_name="grok-4-fast-non-reasoning",
        temperature=0
    )
)

llm_deepseek_adapter = LLMModel(
    name="DeepSeek-V3.1",
    instance=OpenAIClientAdapter(
        client=async_openai_client,
        model_name="DeepSeek-V3.1",
        temperature=0
    )
)

llm_llama_adapter = LLMModel(
    name="Llama-3.3-70B-Instruct",
    instance=OpenAIClientAdapter(
        client=async_openai_client,
        model_name="Llama-3.3-70B-Instruct",
        temperature=0
    )
)

Creando clientes y adaptadores...
Adaptador creado para el modelo: grok-4-fast-non-reasoning
Adaptador creado para el modelo: DeepSeek-V3.1
Adaptador creado para el modelo: Llama-3.3-70B-Instruct


### Test

In [13]:
with open("prompts/human_prompt_no_context.txt", "r", encoding="utf-8") as f:
    human_prompt_no_context = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_no_context = human_prompt_no_context.replace("\n", " ").replace('"', '\\"')
    human_prompt_no_context = human_prompt_no_context.replace("$", "{").replace("%", "}")

with open("prompts/human_prompt_context.txt", "r", encoding="utf-8") as f:
    human_prompt_context = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_context = human_prompt_context.replace("\n", " ").replace('"', '\\"')
    human_prompt_context = human_prompt_context.replace("$", "{").replace("%", "}")

with open("prompts/system_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

In [14]:
score_mapping = {
    "RIP": 0,
    "Bad": 1,
    "Mediocre": 2,
    "Good": 3,
    "Great!": 4,
    "Godly!": 5,
}

In [15]:
def process_deck_rating(rating: str) -> dict:
    rating = rating.strip()
    rating = rating.split(" ")
    ratings = {
        "Attack": score_mapping.get(rating[1], -1),
        "Defense": score_mapping.get(rating[3], -1),
        "Synergy": score_mapping.get(rating[5], -1),
        "Versatility": score_mapping.get(rating[7], -1),
        "F2P score": score_mapping.get(rating[10], -1),
    }
    return ratings

In [16]:
from pathlib import Path
import re
import json

def parse_json_safe(player_cards, llm_response, llm_name):
    player_tag = player_cards.tag[1:]
    deck_cards = player_cards.deck_cards
    deleted_cards = player_cards.deleted_cards or []

    try:
        s = (llm_response or "").strip()

        m = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", s, flags=re.IGNORECASE)
        if m:
            candidate = m.group(1).strip()
        else:
            candidate = s

        first_brace = candidate.find("{")
        first_bracket = candidate.find("[")
        starts = [i for i in (first_brace, first_bracket) if i != -1]
        
        if not starts:
            print(f"! Error de parseo (Usuario: {player_tag}, LLM: {llm_name}): No se encontró JSON.")
            return None

        start = min(starts)
        end = max(candidate.rfind("}"), candidate.rfind("]"))
        
        if end == -1 or end < start:
            print(f"! Error de parseo (Usuario: {player_tag}, LLM: {llm_name}): JSON mal formado.")
            return None
            
        candidate = candidate[start:end+1]
        parsed = json.loads(candidate)

        if isinstance(parsed, dict):
            parsed["user_id"] = player_tag
            parsed["deleted"] = list(map(lambda c: c.name, deleted_cards))
            parsed["original"] = list(map(lambda c: c.name, deck_cards))
            parsed["llm"] = llm_name
            parsed["with_context"] = True
        else:
            parsed = {
                "user_id": player_tag,
                "data": parsed,
                "deleted": list(map(lambda c: c.name, deleted_cards)),
                "llm": llm_name,
                "with_context": True
            }

        original_deck_rating_string = fetch_deck_rating(parsed["original"] + parsed["deleted"])
        selected_deck_rating_string = fetch_deck_rating(parsed["original"] + parsed["seleccion"])

        original_deck_ratings = process_deck_rating(original_deck_rating_string)
        selected_deck_ratings = process_deck_rating(selected_deck_rating_string)

        parsed["original_deck_rating"] = original_deck_ratings
        parsed["selected_deck_rating"] = selected_deck_ratings

        total_original_deck_rating = sum(original_deck_ratings.values())
        total_selected_deck_rating = sum(selected_deck_ratings.values())

        parsed["total_original_deck_rating"] = total_original_deck_rating
        parsed["total_selected_deck_rating"] = total_selected_deck_rating

        parsed["was_improved"] = total_selected_deck_rating >= total_original_deck_rating

        return parsed

    except json.JSONDecodeError as e:
        print(f"X Error JSONDecode (Usuario: {player_tag}, LLM: {llm_name}): {e}")
        return None
    except Exception as e:
        print(f"! Error inesperado (Usuario: {player_tag}, LLM: {llm_name}): {e}")
        return None

In [17]:
def write_prompt_file(string, user_id):
    with open(f"final_prompts/user_{user_id}.txt", "w", encoding="utf-8") as f:
        f.write(string)

In [18]:
async def process_user_recommendation(llm, player_cards, system_prompt_str, human_prompt_template):
    player_tag = player_cards.tag[1:]
    available_cards = player_cards.available_cards
    deck_cards = player_cards.deck_cards

    string_top_decks = "Top mazos por carta:\n"
    for card in deck_cards:
        card_name = card.name
        string_top_decks += f"  {card_name}:\n"
        decks_for_card = top_decks_per_card.get(card_name, [])
        for deck_key in decks_for_card.keys():
            current_deck = decks_for_card[deck_key].split(",")
            shuffle(current_deck)
            string_top_decks += f"    - {','.join(current_deck)}\n"

    try:
        rendered_human_prompt = human_prompt_template.format(
            TOP_DECKS=string_top_decks,
            CARTAS_DISPONIBLES=json.dumps(list(map(lambda c: c.name, available_cards))),
            CARTAS_SELECCIONADAS=json.dumps(list(map(lambda c: c.name, deck_cards)))
        )
        write_prompt_file(rendered_human_prompt, player_tag)
    except Exception as e:
        print(f"! Error al renderizar prompt para usuario {player_tag}: {e}")
        return None

    messages = [
        SystemMessage(content=system_prompt_str),
        HumanMessage(content=rendered_human_prompt),
    ]

    try:
        ai_msg = await llm.instance.ainvoke(messages)
        response_content = ai_msg.content

        parsed_result = parse_json_safe(player_cards, response_content, llm.name)

        if parsed_result:
            print(f"Éxito: Usuario {player_tag} (LLM: {llm.name})")
            fetch_deck_rating(parsed_result)
        
        return parsed_result

    except Exception as e:
        print(f"! Error API (Usuario: {player_tag}, LLM: {llm.name}): {e}")
        return None

In [19]:
def generate_human_prompt(human_prompt_template, top_decks, available_cards, selected_cards):
    rendered_human_prompt = human_prompt_template.format(
        TOP_DECKS=top_decks,
        CARTAS_DISPONIBLES=available_cards,
        CARTAS_SELECCIONADAS=selected_cards
    )
    return rendered_human_prompt

In [20]:
def shuffle_and_remove_cards(player_cards_list, num_to_remove=4):
    updated_player_cards_list = []
    for player_cards in player_cards_list:
        
        available_copy = list(player_cards.available_cards)
        selected_copy = list(player_cards.deck_cards)

        shuffle(available_copy)
        shuffle(selected_copy)

        to_delete = selected_copy[-num_to_remove:]
        selected_for_prompt = selected_copy[:-num_to_remove]

        updated_player_cards = PlayerCards(
            tag=player_cards.tag,
            available_cards=available_copy,
            deck_cards=selected_for_prompt,
            deleted_cards=to_delete
        )
        updated_player_cards_list.append(updated_player_cards)
    
    return updated_player_cards_list


In [21]:
async def run_all_models_async(
    player_cards_list, 
    models_list,
    system_prompt_str, 
    human_prompt_template, 
    batch_size=50, 
    num_batches=None,
    starting_card_number=4
):
    """
    Orquesta la ejecución asíncrona para todos los LLMs y usuarios,
    controlando el tamaño de los lotes.
    """
    selected_player_cards_list = shuffle_and_remove_cards(
        player_cards_list, 
        num_to_remove= 8 - starting_card_number
    )

    if num_batches is not None:
        total_to_process = batch_size * num_batches
        players_to_process = selected_player_cards_list[:total_to_process]
    else:
        players_to_process = selected_player_cards_list

    print(f"Iniciando procesamiento para {len(players_to_process)} jugadores a través de {len(models_list)} LLMs.")

    all_tasks = []
    
    for llm in models_list:
        print(f"Creando tareas para: {llm.name}...")
        for player_cards in players_to_process:
            task = process_user_recommendation(
                llm,
                player_cards, 
                system_prompt_str, 
                human_prompt_template
            )
            all_tasks.append(task)
            
    print(f"Total de tareas creadas: {len(all_tasks)}")
    
    all_results = []
    total_batches = (len(all_tasks) + batch_size - 1) // batch_size
    
    for i in range(0, len(all_tasks), batch_size):
        current_batch_num = (i // batch_size) + 1
        batch_tasks = all_tasks[i:i + batch_size]
        print(f"\n--- Ejecutando Lote {current_batch_num} / {total_batches} (Tamaño: {len(batch_tasks)}) ---")
        
        batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)
        all_results.extend(batch_results)
        await asyncio.sleep(1) 

    print("\n--- Procesamiento de lotes completado. ---")
    return all_results

In [ ]:
import time

models_to_run = [
    llm_gpt3,
    llm_gpt4o,
    llm_gpt5,
    #llm_grok_adapter,
    llm_deepseek_adapter,
    llm_llama_adapter,
]

BATCH_SIZE = 10 
NUM_BATCHES = 1

print("Iniciando la ejecución asíncrona...")

all_final_results = await run_all_models_async(
    player_cards_list=player_cards_list,
    models_list=models_to_run,
    system_prompt_str=system_prompt,
    human_prompt_template=human_prompt_context,
    batch_size=BATCH_SIZE,
    num_batches=NUM_BATCHES
)

print(f"Ejecución finalizada. Resultados totales obtenidos: {len(all_final_results)}")

valid_results = [r for r in all_final_results if r is not None]
print(f"Resultados válidos (no nulos): {len(valid_results)}")

if valid_results:
    df = pd.DataFrame(valid_results)
    
    print("\n--- Conteo de resultados por LLM ---")
    print(df['llm'].value_counts())

    output_filename = f"results/final_results_{time.strftime('%Y%m%d_%H%M%S')}.json"
    df.to_json(output_filename, orient="records", indent=2, force_ascii=False)
    
    print(f"Exito! Todos los resultados guardados en '{output_filename}'")
else:
    print("X No se obtuvieron resultados válidos.")

Iniciando la ejecución asíncrona...
Iniciando procesamiento para 10 jugadores a través de 5 LLMs.
Creando tareas para: gpt-35-turbo...
Creando tareas para: gpt-4o...
Creando tareas para: gpt-5-chat...
Creando tareas para: DeepSeek-V3.1...
Creando tareas para: Llama-3.3-70B-Instruct...
Total de tareas creadas: 50

--- Ejecutando Lote 1 / 5 (Tamaño: 10) ---


In [ ]:
df = pd.read_json(output_filename, orient="records")

In [ ]:
df['correct_selection_count'] = df.apply(
    lambda row: sum(1 for item in row['deleted'] if item in row['seleccion']),
    axis=1
)
df

,seleccion,detalle,resumen,n,user_id,deleted,original,llm,with_context,original_deck_rating,selected_deck_rating,total_original_deck_rating,total_selected_deck_rating,was_improved,correct_selection_count
0,"[Goblin Gang, Inferno Dragon, Mega Knight, Poi...","[{'carta': 'Goblin Gang', 'costo_elixir': '3',...",El mazo se enfoca en un arquetipo de control c...,4,VL0QCY9R8,"[Skeletons, Bats, Mega Knight, Bandit]","[Spear Goblins, Zap, Skeleton Barrel, Firecrac...",gpt-35-turbo,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 3, 'Defense': 5, 'Synergy': 3, 'Ver...",20,19,False,1
1,"[Inferno Dragon, Poison, Ice Golem, Goblin Gang]","[{'carta': 'Inferno Dragon', 'costo_elixir': '...",El mazo se enfoca en un estilo de juego contro...,4,L0092LLGP,"[Skeleton Barrel, Skeletons, Spear Goblins, Fi...","[Mega Knight, Bandit, Zap, Bats]",gpt-35-turbo,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 2, 'Defense': 5, 'Synergy': 1, 'Ver...",20,15,False,0
2,"[Electro Spirit, Fireball, Goblin Gang, Mega M...","[{'carta': 'Electro Spirit', 'costo_elixir': '...",El mazo se enfoca en un arquetipo de ciclo y c...,4,2G22QVP89,"[Electro Spirit, Goblin Cage, Skeleton Barrel,...","[Zappies, Golden Knight, Arrows, Royal Recruits]",gpt-35-turbo,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 1, 'Ver...","{'Attack': 2, 'Defense': 5, 'Synergy': 0, 'Ver...",15,15,False,1
3,"[Goblin Gang, Inferno Tower, Fireball, Mega Mi...","[{'carta': 'Goblin Gang', 'costo_elixir': '3',...",El mazo se enfoca en un estilo de juego de con...,4,UL98CLGP,"[Wall Breakers, Spear Goblins, Bomb Tower, Mag...","[Miner, The Log, Tornado, Knight]",gpt-35-turbo,True,"{'Attack': 4, 'Defense': 5, 'Synergy': 5, 'Ver...","{'Attack': 2, 'Defense': 5, 'Synergy': 3, 'Ver...",16,15,False,0
4,"[Goblin Gang, Zap, Fireball, Mega Minion]","[{'carta': 'Goblin Gang', 'costo_elixir': '3',...",El mazo se enfoca en un estilo de juego contro...,4,RCV90RLVC,"[Skeleton Barrel, The Log, Goblin Gang, Firecr...","[Mega Knight, Poison, Inferno Dragon, Tombstone]",gpt-35-turbo,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 2, 'Ver...","{'Attack': 2, 'Defense': 5, 'Synergy': 1, 'Ver...",17,15,False,1
5,"[Goblin Hut, Fireball, Zappies, Goblin Gang]","[{'carta': 'Goblin Hut', 'costo_elixir': '5', ...",El mazo final se basa en un arquetipo de contr...,4,YC9RRVYYL,"[Zappies, Goblin Cage, Royal Hogs, Arrows]","[Electro Spirit, Flying Machine, Royal Recruit...",gpt-35-turbo,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 2, 'Ver...","{'Attack': 1, 'Defense': 5, 'Synergy': 0, 'Ver...",17,13,False,1
6,"[Miner, Inferno Dragon, Bats, Mega Knight]","[{'carta': 'Miner', 'costo_elixir': 3, 'rol': ...",El mazo se centra en un arquetipo de bait con ...,4,VL0QCY9R8,"[Skeletons, Bats, Mega Knight, Bandit]","[Spear Goblins, Zap, Skeleton Barrel, Firecrac...",gpt-4o,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 5, 'Defense': 5, 'Synergy': 5, 'Ver...",20,22,True,2
7,"[Inferno Dragon, Electro Wizard, Goblin Gang, ...","[{'carta': 'Inferno Dragon', 'costo_elixir': 4...",El mazo final es un arquetipo de bridge spam c...,4,L0092LLGP,"[Skeleton Barrel, Skeletons, Spear Goblins, Fi...","[Mega Knight, Bandit, Zap, Bats]",gpt-4o,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 2, 'Defense': 5, 'Synergy': 1, 'Ver...",20,13,False,0
8,"[Miner, Poison, Knight, Bomb Tower]","[{'carta': 'Miner', 'costo_elixir': '3', 'rol'...",El mazo final es un arquetipo de control con M...,4,LU2QQJU0Y,"[Hog Rider, Executioner, Mega Knight, Minions]","[Firecracker, Musketeer, Electro Wizard, The Log]",gpt-4o,True,"{'Attack': 4, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 3, 'Defense': 5, 'Synergy': 3, 'Ver...",20,16,False,0
9,"[Goblin Barrel, Inferno Dragon, Ice Wizard, Fi...","[{'carta': 'Goblin Barrel', 'costo_elixir': '3...",El mazo final es un arquetipo de bait/control ...,4,999QV8RJU,"[Mega Knight, Cannon, Miner, Wall Breakers]","[Goblin Gang, The Log, Bats, Zap]",gpt-4o,True,"

In [ ]:
df['correct_selection_count'].groupby(df['llm']).value_counts()

llm            correct_selection_count
DeepSeek-V3.1  0                          7
               1                          2
gpt-35-turbo   1                          4
               0                          2
gpt-4o         0                          6
               2                          2
               1                          1
               3                          1
Name: count, dtype: int64

In [ ]:
res = (
    df.assign(wi=lambda d: d["was_improved"].astype(bool))  # robusto si viene como str/0/1
      .groupby("llm", as_index=False)
      .agg(total_mazos=("llm", "size"),
           mazos_mejorados=("wi", "sum"))
)
print(res)

             llm  total_mazos  mazos_mejorados
0  DeepSeek-V3.1            9                2
1   gpt-35-turbo            6                0
2         gpt-4o           10                2
